In [ ]:
#testing
from flask import Flask, render_template, request
import pyodbc
from webscraping import automate_web_scraping

app = Flask(__name__)

try:
    # Database connection
    conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                          'Server=(local);'
                          'Database=MY_CUSTOM_BOT;'
                          'Trusted_Connection=yes;')
except pyodbc.Error as e:
    print("Error connecting to the database:", e)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/search', methods=['POST'])
def search():
    try:
        query = request.form['query']
        automate_web_scraping(query)

        cursor = conn.cursor()

        # Query 1: Remove duplicate rows
        query1 = """
        WITH RankedRows AS (
            SELECT *,
                   ROW_NUMBER() OVER (PARTITION BY QueryID, URLID, SearchTerm ORDER BY Frequency DESC) AS RowNum
            FROM SearchTermFrequency
        )
        DELETE FROM RankedRows
        WHERE RowNum > 1;
        """

        # Execute Query 1
        cursor.execute(query1)

        # Commit the transaction 
        conn.commit()

        # Query 2: Retrieve URLs associated with the last query
        query2 = """
        WITH LastQueryID AS (
            SELECT TOP 1 QueryID
            FROM SearchQueries
            ORDER BY QueryID DESC
        )
        SELECT URLs.URL, SUM(SearchTermFrequency.Frequency) AS TotalFrequency
        FROM SearchTermFrequency
        JOIN URLs ON SearchTermFrequency.URLID = URLs.URLID
        WHERE SearchTermFrequency.QueryID = (SELECT QueryID FROM LastQueryID)
        GROUP BY URLs.URL, SearchTermFrequency.URLID
        ORDER BY TotalFrequency DESC;
        """

        # Execute Query 2
        cursor.execute(query2)

        # Fetch all results
        results = cursor.fetchall()

        # Extract URLs from the results
        urls = [row[0] for row in results]

        # Render template with results
        return render_template('search_results.html', results=urls)

    except Exception as e:
        print("Error occurred:", e)

if __name__ == '__main__':
    app.run()